In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
DetailOutageTicket = pd.read_excel("TTOP 12 July Up2.xlsx")

In [ ]:
# DetailOutage = DetailOutageTicket[((DetailOutageTicket['Impact']!= 'LEVEL4') & (DetailOutageTicket['Impact']!= 'MINOR'))]
# DetailOutage = DetailOutageTicket[((DetailOutageTicket['Impact']!= 'LEVEL3') & (DetailOutageTicket['Impact']!= 'LEVEL4') & (DetailOutageTicket['Impact']!= 'MINOR'))]

DetailOutage = DetailOutageTicket

In [ ]:
import re

pat_name = '\\A[A-Z]{2}[1-9]*[GLU]{0,3}[0-9]{4}'; #'[A-F]{1}'
DetailOutage['CellName'] = DetailOutage['CellName'].astype(str);
DetailOutage['CellName'] = DetailOutage['CellName'].str.upper();
DetailOutage['Sector'] = DetailOutage['CellName'].apply(lambda x:  ''.join(re.findall(pat_name, x)));
DetailOutage = DetailOutage[DetailOutage['Sector'] != ''];
DetailOutage['Site']=np.where(len(DetailOutage['Sector']) > 7, DetailOutage['Sector'].str[:8], DetailOutage['Sector'].str[:6]);
DetailOutage.drop(columns = ['Sector'], inplace = True)

In [ ]:
DetailOutage.rename(columns={'Province':'Province Index'}, inplace = True)

DetailCity = pd.read_excel(r'Cell Reff/BTS Node 2020.xlsx', sheet_name='Sheet2')
DetailPro = pd.read_excel(r'Cell Reff/BTS Node 2020.xlsx', sheet_name='Sheet3')

# Detail.rename(columns={'شهر':'City', 'استان':'Province'}, inplace = True)
# DetailCity.drop_duplicates(subset =["Location"],keep = 'first', inplace = True)

DetailOutage = DetailOutage.merge(DetailCity[['Location','City']], on = ['Location'], how = 'left')
DetailOutage = DetailOutage.merge(DetailPro[['Province Index', 'Province']], on = ['Province Index'], how = 'left')

In [ ]:
Outage_Grouping = DetailOutage[['CellName', 'Site', 'Province Index', 'NE', 'Location','Tech','StartOfOutage','Duration',\
                                'EndOfOutage', 'Province', 'City', 'Impact','ENTRY_TicketNumber','TicketTitle',\
                                'TicketClosureType', 'RootCause','EventEnd','Health_Check','Exclude_Cause']]

In [ ]:
Outage_Grouping['StartOfOutage'] = pd.to_datetime(Outage_Grouping['StartOfOutage'])
Outage_Grouping['EndOfOutage'] = pd.to_datetime(Outage_Grouping['EndOfOutage'])
Outage_Grouping.loc[Outage_Grouping['Tech']=='4G','NE'] = Outage_Grouping.loc[Outage_Grouping['Tech']=='4G','NE'] + '_LTE'

In [ ]:
Outage_Grouping.rename(columns={'Duration': 'Down_Time(Seconds)', 'NE': 'BSC-RNC'}, inplace = True)

In [ ]:
# Outage_Grouping.to_excel('TTOP 12 July.xlsx', index = False, encoding = 'utf-8-sig')

### Grouping By Province

In [ ]:
Outage_Grouping.sort_values(by = ['Province Index','StartOfOutage','Down_Time(Seconds)'], inplace = True)
Outage_Grouping = Outage_Grouping.reset_index(drop=True)

In [ ]:
outage = Outage_Grouping.groupby(['Province','StartOfOutage'], as_index = True)['Site'].nunique().reset_index().reset_index()

In [ ]:
outage.rename(columns={'index': 'GroupNumPro','Site': 'CountOfSitePerProvince'}, inplace = True) 
Outage_Grouping = Outage_Grouping.merge(outage, on = ['Province', 'StartOfOutage'], how = 'left') 

In [ ]:
Outage_Grouping1 = Outage_Grouping[Outage_Grouping['Tech'] != '4G'] #BSC-RNC
Outage_Grouping2 = Outage_Grouping[Outage_Grouping['Tech'] == '4G'] #EnodB

### Grouping by BSC-RNC sites

In [ ]:
Outage_Grouping1.sort_values(by = ['BSC-RNC','StartOfOutage','Down_Time(Seconds)'], inplace = True)
Outage_Grouping1 = Outage_Grouping1.reset_index(drop=True)

In [ ]:
outage1 = Outage_Grouping1.groupby(['BSC-RNC','StartOfOutage'], as_index = True)['Site'].nunique().reset_index().reset_index()

In [ ]:
outage1.rename(columns={'index': 'GroupNum','Site': 'CountOfSitePerGroup'}, inplace = True) 

In [ ]:
Outage_Grouping1 = Outage_Grouping1.merge(outage1, on = ['BSC-RNC', 'StartOfOutage'], how = 'left') 

In [ ]:
SiteCount_BSCRNC = pd.read_excel("SITE COUNT (BSC-RNC-City-Province)_2G3G4G.xlsx", sheet_name='BSC-RNC')

In [ ]:
Outage_Grouping1 = Outage_Grouping1.merge(SiteCount_BSCRNC[['BSC-RNC', 'SITE_COUNT(30%)']], on = ['BSC-RNC'], how = 'left')

### Grouping according to provincial 4G sites:

In [ ]:
Outage_Grouping2.sort_values(by = ['Province','StartOfOutage','Down_Time(Seconds)'], inplace = True)
Outage_Grouping2 = Outage_Grouping2.reset_index(drop=True)

In [ ]:
outage2 = Outage_Grouping2.groupby(['Province','StartOfOutage'], as_index = True)['Site'].nunique().reset_index().reset_index()

In [ ]:
outage2.rename(columns={'index': 'GroupNum','Site': 'CountOfSitePerGroup'}, inplace = True) 
Outage_Grouping2 = Outage_Grouping2.merge(outage2, on = ['Province', 'StartOfOutage'], how = 'left') 

In [ ]:
Outage_Grouping2['GroupNum'] = Outage_Grouping2['GroupNum']  + Outage_Grouping1['GroupNum'].max() + 1

In [ ]:
SiteCount_Province = pd.read_excel("SITE COUNT (BSC-RNC-City-Province)_2G3G4G.xlsx", sheet_name='Province')

In [ ]:
Outage_Grouping2 = Outage_Grouping2.merge(SiteCount_Province[['Province Index', 'SITE_COUNT(30%)']], on = ['Province Index'], how = 'left')

In [ ]:
Ticket_Outage_Grouping = pd.concat([Outage_Grouping1, Outage_Grouping2], sort = True )

In [ ]:
Ticket_Outage_Grouping = Ticket_Outage_Grouping[['CellName', 'Site', 'Location','Tech', 'BSC-RNC', 'Province Index','Province',\
                                                 'City','GroupNumPro','GroupNum','CountOfSitePerGroup','CountOfSitePerProvince', 'SITE_COUNT(30%)',\
                                                 'StartOfOutage','Down_Time(Seconds)','EndOfOutage', 'Impact','ENTRY_TicketNumber','TicketTitle',\
                                                 'TicketClosureType', 'RootCause','EventEnd','Health_Check','Exclude_Cause']]

In [ ]:
Ticket_Outage_Grouping.to_excel('Ticket_Outage_Grouping12July.xlsx', index = False, encoding = 'utf-8-sig')

In [ ]:
Ticket_Outage_Grouping.columns